In [2]:
import tensorflow.compat.v1 as tf
import warnings
tf.disable_v2_behavior()
tf.set_random_seed(777)  # for reproducibility
warnings.filterwarnings('ignore')
print(tf.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
2.4.0


In [3]:
import numpy as np

In [4]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

In [5]:
X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name="weight_1")
    b1 = tf.Variable(tf.random_normal([2]), name="bias_1")
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)


with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="weight_2")
    b2 = tf.Variable(tf.random_normal([1]), name="bias_2")
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)

In [6]:
# cost/loss function
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [7]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [8]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

In [9]:
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./teslogs/xor_logs_r0_01")
writer.add_graph(sess.graph)  # Show the graph
# Initialize TensorFlow variables
sess.run(tf.global_variables_initializer())

In [10]:
for step in range(10001):
    _, summary, cost_val = sess.run(
        [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
    )
    writer.add_summary(summary, global_step=step)
    if step % 100 == 0:
        print(step, cost_val)


0 0.9458089
100 0.6640444
200 0.6141374
300 0.53789586
400 0.34122443
500 0.22424422
600 0.1624983
700 0.12438147
800 0.09863862
900 0.08028249
1000 0.066680945
1100 0.056297295
1200 0.048175525
1300 0.04169313
1400 0.036429457
1500 0.032091863
1600 0.02847157
1700 0.02541596
1800 0.022811553
1900 0.020572213
2000 0.018631807
2100 0.016938727
2200 0.015452113
2300 0.014139455
2400 0.012974378
2500 0.011935441
2600 0.01100502
2700 0.010168565
2800 0.009413847
2900 0.008730612
3000 0.008110165
3100 0.007545186
3200 0.0070294095
3300 0.006557282
3400 0.0061241705
3500 0.005726118
3600 0.005359417
3700 0.005021061
3800 0.00470829
3900 0.0044187256
4000 0.0041501583
4100 0.0039008774
4200 0.0036689956
4300 0.0034532128
4400 0.0032521114
4500 0.0030643642
4600 0.0028890371
4700 0.00272512
4800 0.002571695
4900 0.00242804
5000 0.0022933276
5100 0.0021669418
5200 0.0020482962
5300 0.0019368498
5400 0.0018320628
5500 0.0017334248
5600 0.00164068
5700 0.001553304
5800 0.0014709373
5900 0.0013933

In [11]:
# Accuracy report
h, p, a = sess.run(
    [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
)

print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")



Hypothesis:
[[1.5473366e-04]
 [9.9989766e-01]
 [9.9972683e-01]
 [1.4516711e-04]] 
Predicted:
[[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
1.0
